In [ ]:
#imports
import random
import numpy as np
import time
import pandas as pd
import math
import itertools
from collections import Counter
from itertools import permutations
from sympy import factorint


In [ ]:
#Makes sure the given profile is valid
def parameter_check(profile):

  #assume the following
  sumTo12 = True
  allPositives = True
  allInts = True

  #check each individually:
  #ensure 12 cards
  if (sum(profile)) != 12:
    sumTo12 = False
  #check for negatives
  for numCost in profile:
    if numCost < 0:
      allPositives = False
  #ensure integers
  for numCost in profile:
    if not isinstance(numCost, int):
      allInts = False

  #make an error message
  errorMessage = ""
  if not sumTo12:
    errorMessage += "All inputs must add to 12. "
  if not allPositives:
    errorMessage += "All numbers must be 0 or greater. "
  if not allInts:
    errorMessage += "All numbers must be integers. "

  #raise error if necessary
  if len(errorMessage) != 0:
    raise ValueError(errorMessage)


In [ ]:
primes = np.array([2, 3, 5, 7, 11, 13])


In [ ]:
#Converts a energy-count array into a single integer using prime-factorization encoding
def to_primes(array):
  components = np.array(array) * primes
  return np.prod(components[components != 0])


In [ ]:
#This gives strategy of what cost combibnations to play
#always_highest primarily tries to play the least amount of cards, and secondarily plays the combination with the largest maximum card
#this is the strategy lerio used

#STRATEGIES OTHER THAN ALWAYS_HIGHEST WILL CURRENTLY NOT WORK PROPERLY WITH CODE AND NEED TO BE REWRITTEN/CONVERTED INTO COUNT VECTORS

def strategy(cost, strat):
  match strat:
    case "always_highest":
      match cost:
        case 1:
          return [[1, 0, 0, 0, 0, 0]]
        case 2:
          return [[0, 1, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0]]
        case 3:
          return [[0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0], [3, 0, 0, 0, 0, 0]]
        case 4:
          return [[0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0], [0, 2, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0], [4, 0, 0, 0, 0, 0]]
        case 5:
          return [[0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0], [2, 0, 1, 0, 0, 0], [1, 2, 0, 0, 0, 0], [3, 1, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0]]
        case 6:
          return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 0, 0], [2, 0, 0, 1, 0, 0], [0, 0, 2, 0, 0, 0], [1, 1, 1, 0, 0, 0], [3, 0, 1, 0, 0, 0], [0, 3, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0], [4, 1, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0]]
        case _:
          raise ValueError("strategy() input invalid")
    case "lowest_num_cards":
      match cost:
        case 1:
          return [[1]]
        case 2:
          return [[2], [1, 1]]
        case 3:
          return [[3], [2, 1], [1, 1, 1]]
        case 4:
          return [[4], [3, 1], [2, 2], [2, 1, 1], [1, 1, 1, 1]]
        case 5:
          return [[5], [4, 1], [3, 2], [3, 1, 1], [2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1]]
        case 6:
          return [[6], [5, 1], [4, 2], [3, 3], [4, 1, 1], [3, 2, 1], [2, 2, 2], [3, 1, 1, 1], [2, 2, 1, 1], [2, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]
        case _:
          raise ValueError("strategy() input invalid")
    case "lowest_number_cards_balanced":
      match cost:
        case 1:
          return [[1]]
        case 2:
          return [[2], [1, 1]]
        case 3:
          return [[3], [2, 1], [1, 1, 1]]
        case 4:
          return [[4], [2, 2], [3, 1], [2, 1, 1], [1, 1, 1, 1]]
        case 5:
          return [[5], [3, 2], [4, 1], [2, 2, 1], [3, 1, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1]]
        case 6:
          return [[6], [3, 3], [4, 2], [5, 1], [2, 2, 2], [3, 2, 1], [4, 1, 1], [2, 2, 1, 1], [3, 1, 1, 1], [2, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]
        case _:
          raise ValueError("strategy() input invalid")
    case "PowerValue":
    #Assigns value of play based on in-game baseline values. Corresponding values are [2, 3, 4, 6, 9, 12]
      match cost:
        case 1:
          return [[1]]
        case 2:
          return [[1, 1], [2]]
        case 3:
          return [[3], [2, 1], [1, 1, 1]]
        case 4:
          return [[1, 1, 1, 1], [2, 1, 1], [4], [3, 1], [2, 2]]
        case 5:
          return [[1, 1, 1, 1, 1], [5], [2, 1, 1, 1], [4, 1], [3, 1, 1], [2, 2, 1], [3, 2]]
        case 6:
          return [[6], [1, 1, 1, 1, 1, 1], [5, 1], [2, 1, 1, 1, 1], [4, 1, 1], [3, 1, 1, 1], [4, 2], [3, 2, 1], [3, 3], [2, 2, 1]]
          #        12         12             11         11             10           10          9        9        8         8
          # Above are the summed values of case 6's plays
        case _:
          raise ValueError("strategy() input invalid")

#This is where you can change strategy
strat = "always_highest"

plays1 = [to_primes(strategy(1, strat)[0])]

plays2 = [to_primes(strategy(2, strat)[0]), to_primes(strategy(2, strat)[1])]
plays2.extend(plays1)

plays3 = [to_primes(strategy(3, strat)[0]), to_primes(strategy(3, strat)[1]), to_primes(strategy(3, strat)[2])]
plays3.extend(plays2)

plays4 = [to_primes(strategy(4, strat)[0]), to_primes(strategy(4, strat)[1]), to_primes(strategy(4, strat)[2]), to_primes(strategy(4, strat)[3])]
plays4.extend(plays3)

plays5 = [to_primes(strategy(5, strat)[0]), to_primes(strategy(5, strat)[1]), to_primes(strategy(5, strat)[2]), to_primes(strategy(5, strat)[3]), to_primes(strategy(5, strat)[4])]
plays5.extend(plays4)

plays6 = [to_primes(strategy(6, strat)[0]), to_primes(strategy(6, strat)[1]), to_primes(strategy(6, strat)[2]), to_primes(strategy(6, strat)[3]), to_primes(strategy(6, strat)[4]), to_primes(strategy(6, strat)[5])]
plays6.extend(plays5)

plays = [plays1, plays2, plays3, plays4, plays5, plays6]


In [ ]:
#Has 4 positions that each follow the same rule: increment until you reach a new number or reach the end,
#then move previous cursor forward one, and start immediately in front of it.
#this only outputs groupings of 4

def list_deals(deck):
  position1 = 0
  position2 = 1
  position3 = 2
  position4 = 3
  card1 = deck[position1]
  card2 = deck[position2]
  card3 = deck[position3]
  card4 = deck[position4]
  deals = []
  while position1 < len(deck) - 3:
    card1 = deck[position1]
    while position2 < len(deck) - 2:
      card2 = deck[position2]
      while position3 < len(deck) - 1:
        card3 = deck[position3]
        while position4 < len(deck):
          card4 = deck[position4]
          deals.append([card1, card2, card3, card4])
          position4 += 1
          while position4 < len(deck) and deck[position4] == deck[position4 - 1]:
            position4 += 1
        position3 += 1
        while position3 < len(deck) - 1 and deck[position3] == deck[position3 - 1]:
            position3 += 1
        position4 = position3 + 1
      position2 += 1
      while position2 < len(deck) - 2 and deck[position2] == deck[position2 - 1]:
          position2 += 1
      position3 = position2 + 1
      position4 = position3 + 1
    position1 += 1
    while position1 < len(deck) - 3 and deck[position1] == deck[position1 - 1]:
        position1 += 1
    position2 = position1 + 1
    position3 = position2 + 1
    position4 = position3 + 1
  return deals


In [ ]:
def list_turn_draws(deck):
    return [list(p) for p in set(permutations(deck, 5))]


In [ ]:
#finds weights of a deal (number of times a deal happens)
def find_comb_weights(deck_profile, deal_profiles):
  weights = []
  for deal_profile in deal_profiles:
    weight = 1
    for i in range(6):
      if deal_profile[i] != 0:
        weight *= math.comb(deck_profile[i], deal_profile[i])
    weights.append(weight)
  return weights


In [ ]:
#finds weights of a draw (number of times a draw happens)
def find_perm_weight(deck_profile, draw_profile):
  weight = 1
  for i in range(6):
    if draw_profile[i] != 0:
      weight *= math.perm(deck_profile[i], draw_profile[i])
  return weight


In [ ]:
#if given profile of [3, 3, 3, 2, 1, 0], makes deck of [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5]
def create_deck(profile):

  deck = []
  cost = 0

  for numCost in profile:
    cost += 1
    for i in range(numCost):
      deck.append(cost)

  return deck


In [ ]:
#takes an array of cards and converts them into a profile
#if given cards [1, 1, 2, 3], makes profile of [2, 1, 1, 0, 0, 0]

def convert_to_profile(deals):
  deal_profiles = []
  for deal in deals:
    profile = [0, 0, 0, 0, 0, 0]
    for card in deal:
      profile[card - 1] += 1
    deal_profiles.append(np.array(profile))
  return deal_profiles

def convert_to_profile2(deal):
    # print("Deal: ", deal)
    profile = [0, 0, 0, 0, 0, 0]
    for card in deal:
      # print("Card: ", card)
      profile[card - 1] += 1
    return profile


In [ ]:
np_turns = np.array([1, 2, 3, 4, 5, 6])
turns = [1, 2, 3, 4, 5, 6]


In [ ]:
#plays a series of games
#hand in profile, deck in expanded
def play(hand, deck):
  board = 1
  hand = to_primes(hand)
  for turn in turns:
    if (turn != 1):
      hand *= primes[deck.pop() - 1]
    for play in plays[turn - 1]:
      test = hand / play
      if test.is_integer():
        hand = test
        board *= play
        break
  factor_dict = factorint(int(board))
  count = [factor_dict.get(p, 0) for p in primes]
  return np.dot(count, np_turns)


In [ ]:
#Calls many methods to find efficiency
def find_efficiency(deck_profile):
  parameter_check(deck_profile)
  deck = create_deck(deck_profile)
  deals = list_deals(deck)
  deal_profiles = np.array(convert_to_profile(deals))
  deal_weights = find_comb_weights(deck_profile, deal_profiles)
  deck_profile = np.array(deck_profile)
  average_deal_energies = np.empty(len(deals))
  deal_num = 0


  for deal_profile in deal_profiles:
    in_deck_profile = deck_profile - deal_profile
    turn_draws = list_turn_draws(create_deck(in_deck_profile))
    turn_draw_weights = np.zeros(len(turn_draws))
    turn_draw_energy = np.zeros(len(turn_draws))
    turn_draw_index = 0
    for turn_draw in turn_draws:
      turn_draw_weights[turn_draw_index] = find_perm_weight(in_deck_profile, convert_to_profile2(turn_draw))
      turn_draw_energy[turn_draw_index] = turn_draw_weights[turn_draw_index] * play(deal_profile, turn_draw)
      turn_draw_index += 1
    average_deal_energies[deal_num] = np.sum(turn_draw_energy) / np.sum(turn_draw_weights)
    deal_num += 1

  weighted_deal_energy = np.dot(average_deal_energies, deal_weights)
  average_energy = weighted_deal_energy / sum(deal_weights)

  return average_energy


In [ ]:
#generate all profiles

def generate_lists(total, length, current_list=None, result=None):
    if current_list is None:
        current_list = []

    if result is None:
        result = []

    if total == 0 and len(current_list) == length:
        result.append(current_list[:])  # Append a copy of the current list
        return

    if total < 0 or len(current_list) >= length:
        return

    for i in range(0, 13):  # including 0 as a valid number
        generate_lists(total - i, length, current_list + [i], result)

    return result


In [ ]:
#test all profiles
start_time = time.time()

combinations = generate_lists(12, 6)
data = {'Profile': combinations, 'Efficiency': 0.0}

# Create the DataFrame
df = pd.DataFrame(data)

row = 0
batch_start_time = time.time()

for combo in combinations:
  NRGsum = 0
  NRGsum += find_efficiency(combo)
  df.loc[row, 'Efficiency'] = NRGsum
  if row % 100 == 0 and row != 0:
    batch_end_time = time.time()
    batch_elapsed_time = batch_end_time - batch_start_time
    print(f"Time taken for rows {row-100} to {row-1}: {batch_elapsed_time:.2f} seconds")
    batch_start_time = batch_end_time
  row += 1
df['Efficiency'] = df['Efficiency'] / (21)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Time taken for rows 0 to 99: 2.57 seconds
Time taken for rows 100 to 199: 8.35 seconds
Time taken for rows 200 to 299: 14.46 seconds
Time taken for rows 300 to 399: 11.74 seconds
Time taken for rows 400 to 499: 4.95 seconds
Time taken for rows 500 to 599: 16.92 seconds
Time taken for rows 600 to 699: 38.64 seconds
Time taken for rows 700 to 799: 25.43 seconds
Time taken for rows 800 to 899: 12.21 seconds
Time taken for rows 900 to 999: 49.03 seconds
Time taken for rows 1000 to 1099: 38.61 seconds
Time taken for rows 1100 to 1199: 26.86 seconds
Time taken for rows 1200 to 1299: 50.62 seconds
Time taken for rows 1300 to 1399: 22.21 seconds
Time taken for rows 1400 to 1499: 31.00 seconds
Time taken for rows 1500 to 1599: 23.64 seconds
Time taken for rows 1600 to 1699: 10.90 seconds
Time taken for rows 1700 to 1799: 4.23 seconds
Time taken for rows 1800 to 1899: 6.28 seconds
Time taken for rows 1900 to 1999: 29.46 seconds
Time taken for rows 2000 to 2099: 37.20 seconds
Time taken for rows 

In [ ]:
df.to_excel('Combinatorics_v4.1.xlsx', index=False)
